In [348]:
import pandas as pd
import re


df = pd.read_excel('provider_list.xlsx', sheet_name = "Provider_by_Group_NT")
addresses = df["Unnamed: 6"][1:]
addresses.reset_index(drop=True, inplace=True)
ind = pd.read_parquet("index.parquet")

/home/yunseok/miniconda3/envs/jupyter/lib/python3.8/site-packages/openpyxl/worksheet/header_footer.py:48: UserWarning: Cannot parse header or footer so it will be ignored
  warn("""Cannot parse header or footer so it will be ignored""")


In [2]:
addresses[0]

'9, George Street, North Strathfield, NSW 2137'

In [399]:
def match_address(adr):
    street_code_dict = {
        'ALLY': 'ALLEY',
         'ARC': 'ARCADE',
         'AVE': 'AVENUE',
         'AV': 'AVENUE',
         'BVD': 'BOULEVARD',
         'BYPA': 'BYPASS',
         'CCT': 'CIRCUIT',
         'CL': 'CLOSE',
         'CRN': 'CORNER',
         'CT': 'COURT',
         'CRES': 'CRESCENT',
         'CDS': 'CUL-DE-SAC',
         'DR': 'DRIVE',
         'ESP': 'ESPLANADE',
         'GRN': 'GREEN',
         'GR': 'GROVE',
         'HWY': 'HIGHWAY',
         'JNC': 'JUNCTION',
         'LANE': 'LANE',
         'LINK': 'LINK',
         'MEWS': 'MEWS',
         'PDE': 'PARADE',
         'PL': 'PLACE',
         'RDGE': 'RIDGE',
         'RD': 'ROAD',
         'SQ': 'SQUARE',
         'ST': 'STREET',
         'TCE': 'TERRACE'
    }
    def split_address(adr):
        while True:
            match = re.search("[0-9]+[a-zA-Z,]*\s", adr)
            if not match: break
            pos = match.span()[1]    
            adr = adr[pos:]
        return adr.upper()
    clean_adr = re.sub(r"((?:NSW|VIC|QLD|TAS|WA|SA|NT|ACT))[,\s]*([0-9]{4})$", r"\1 \2",adr)
    clean_adr = clean_adr.replace(",", "").strip()
    clean_adr = split_address(clean_adr)
    match = re.search("\s((?:NSW|VIC|QLD|TAS|WA|SA|NT|ACT))\s([0-9]{4})$", clean_adr)
    # find state and post code
    if not match: return clean_adr
    state, postcode = match.group(1), match.group(2)
    # filter index file based on post code and state
    df = ind[(ind.POSTCODE == postcode) & (ind.STATE == state)]
    if df.empty: return clean_adr
    # filter df based on states
    suburbs = [suburb for suburb in df.LOCALITY_NAME.unique() if suburb in clean_adr]
    # suburb exists within the state
    index = df.LOCALITY_NAME.str.contains('|'.join(suburbs))
    # find street and street code from clean_adr
    match = re.search(f"(?:{'|'.join(suburbs)})\s+{state}\s+{postcode}", clean_adr)
    if not match: return clean_adr
    street_string = clean_adr[:match.span()[0]]
    if any(index):
        # filter by suburbs if suburb matches
        temp_df = df[index]
        street_name = [street_name for street_name in temp_df.STREET_NAME.unique() if street_name in street_string] 
        # if street name does not exist in the data frame, suburbs maybe incorrect
        if street_name:
            df = temp_df[temp_df.STREET_NAME.isin(street_name)] 
        else:
            # find within the same post code and state again for street name
            street_name = [street_name for street_name in df.STREET_NAME.unique() if street_name in street_string]
            df = df[df.STREET_NAME.isin(street_name)] 
    if df.empty: return clean_adr  
    # search for street name and code
    match = re.search(
        f"(?:({'|'.join(df.STREET_NAME.unique())}))\s(.*)\s((:?{'|'.join(df.LOCALITY_NAME.unique())}))", 
        clean_adr)
    if not match: 
        match = re.search(
            f"(?:({'|'.join(df.STREET_NAME.unique())}))\s(\S*)\s", 
            clean_adr)
        if not match: return clean_adr
        street_name = match.group(1)
        street_code = match.group(2)
        match = re.search(f'{street_code}\s(.+)\s{state}', clean_adr)
        # suburb maybe incorrectly entered
        suburb = match.group(1) if match else None      
    else:
        street_name, street_code, suburb = (match.group(i+1) for i in range(3))
    if suburb:
        # filter based on street and suburb name
        index = (df.STREET_NAME == street_name) & (df.LOCALITY_NAME ==suburb)
        # if no exact match, try to find matching string
        if not any(index): index = (df.STREET_NAME.str.contains(street_name)) & (df.LOCALITY_NAME.str.contains(suburb))
        if(any(index)):
            df = df[index]
        else:
            # if there is no matching suburbs use street name only to filter
            df = df[(df.STREET_NAME.str.contains(street_name))]
    else:
        df = df[(df.STREET_NAME == street_name)]
    if df.empty: return clean_adr
    if df.shape[0] == 1: return df.ADDRESS.iloc[0]          
    # covert street code to long form postcode
    street_code = street_code_dict[street_code] if street_code in street_code_dict else street_code
    df = df[df.STREET_TYPE_CODE == street_code]
    if df.empty: return clean_adr
    if df.shape[0] == 1: return df.ADDRESS.iloc[0]
    return clean_adr

In [394]:
match_address("GUILDFORD RD GUILDFORD NSW 2161")

'GUILDFORD ROAD GUILDFORD NSW 2161'

In [400]:
# match_address(addresses[1089]
x = match_address('24 malsbury rd, Wahroonga nsw 2076')
x

'MALSBURY ROAD NORMANHURST NSW 2076'

In [396]:
def split_address(address):
    adr = address
    while True:
        match = re.search("[0-9]+[a-zA-Z,]*\s", adr)
        if not match: break
        pos = match.span()[1]    
        adr = adr[pos:]
        
    street_no = address[:-len(adr)]
    return street_no.upper().strip(', '), adr.upper()

In [397]:
split_address("Unit 4, 24, malsbury road hornsby nsw 2077")

('UNIT 4, 24', 'MALSBURY ROAD HORNSBY NSW 2077')

In [401]:
adr = addresses[0:10000].apply(match_address)
non_match = adr[~adr.isin(ind.ADDRESS)]
non_match.shape[0]

444

In [402]:
non_match

32                 JAMES STREET TOOWOOMBA QLD 4350
86                              DONCASTER VIC 3108
127               THE CRESCENT KINGSGROVE NSW 2208
144                                   CONFIDENTIAL
161                                   CONFIDENTIAL
                           ...                    
9940                    ESPLANADE URANGAN QLD 4655
9969                     NORTH PARRAMATTA NSW 2151
9975    THE LAKES BOULEVARDE SOUTH MORANG VIC 3752
9977                                  CONFIDENTIAL
9995           HOSKINGS STREET CRANEBROOK NSW 2749
Name: Unnamed: 6, Length: 444, dtype: object

In [390]:
ind[ind.STREET_NAME.str.contains('DAWSON') & ind.LOCALITY_NAME.str.contains('BALLARAT ')]

,IDX,STREET_NAME,STREET_TYPE_CODE,LOCALITY_NAME,STATE,POSTCODE,FILE_NAME,ADDRESS_COUNT,ADDRESS
323198,323199,DAWSON,STREET,BALLARAT CENTRAL,VIC,3350,VIC-3.parquet,119,DAWSON STREET BALLARAT CENTRAL VIC 3350
